In [13]:
# import libraries 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:
# Function to extract Product Title
def get_title(soup):
    try:
        # Try to find an HTML element with the id "productTitle" using the find method of the BeautifulSoup object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Extract the text of the title element and assign it to a variable
        title_value = title.text
        
        # Remove leading and trailing whitespace from the title text and assign the result to a variable
        title_string = title_value.strip()
    except AttributeError:
        # If the title element can't be found or doesn't have a text attribute, assign an empty string to the title variable
        title_string = ""
    
    # Return the title string
    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        # Try to find an HTML element with the id "price_inside_buybox" using the find method of the BeautifulSoup object
        price = soup.find("span", attrs={'id':'price_inside_buybox'}).text
        
    except:
        # If the above step raises an exception, try to find an HTML element with the class "a-offscreen"
        price = soup.find("span", attrs={'class':'a-offscreen'}).text

    # Return the price (or None if no price was found)
    return price



# Function to extract Product Rating
def get_rating(soup):
    try:
        # Try to find an HTML element with the class "a-icon a-icon-star a-star-4-5" using the find method of the BeautifulSoup object
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).text
        
    except:
        # If the above step raises an exception, try to find an HTML element with the same class attribute as in the try block
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).text

    # Return the rating (or None if no rating was found)
    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        # Try to find an HTML element with the id "acrCustomerReviewText" using the find method of the BeautifulSoup object
        # Extract the text of the element and strip whitespace from the resulting string
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        # If the above step raises an exception, set review_count to an empty string
        review_count = ""

    # Return the review count (or an empty string if no review count was found)
    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        # Try to find an HTML element with the id "availability" using the find method of the BeautifulSoup object
        # Then find the first <span> element within that element and extract the text, removing any leading or trailing whitespace characters
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        # If the above step raises an exception, set available to "Not Available"
        available = "Not Available"

    # Return the availability status (or "Not Available" if no availability status was found)
    return available


In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    # webpage URL
    URL = "https://www.amazon.com/s?k=laptop&crid=GYZPZZ1UDJU7&sprefix=laptop%2Caps%2C314&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazonweb_data.csv", header=True, index=False)

In [ ]:
amazon_df

In [ ]:
d